In [48]:
import pandas

ud_train = pandas.read_csv(r'C:\Users\jyellin\re_1\Code\path_to_re\train-ud_paths_per_relation.csv')

In [51]:
ud_train.groupby(['relation','path'])['id'].count().unstack(fill_value=0).T
ud_train_stats = ud_train.groupby(['relation','path'])['id'].count().reset_index()
ud_train_stats = ud_train_stats.rename(columns={'id': 'count'}).sort_values(['relation','count'], ascending=[True,False]).reset_index(drop=True)

In [52]:
paths = ud_train_stats['path'].unique().tolist()
path_to_index = {k: v for v, k in enumerate(paths)}

relations = ud_train_stats['relation'].unique().tolist()
relation_to_index = {k: v for v, k in enumerate(relations)}

In [53]:
import numpy

matrix = numpy.zeros((len(relations), len(paths)))

for index, row in ud_train_stats.iterrows():
    relation_index = relation_to_index[row['relation']]
    path_index = path_to_index[row['path']]
    
    matrix[relation_index, path_index] = row['count']

In [54]:
from sklearn.preprocessing import StandardScaler
matrix_std = StandardScaler().fit_transform(matrix)

In [55]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
fitted = pca.fit_transform(matrix_std)

In [56]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [57]:
data = []

for relation_index, relation in enumerate(relations):

    trace = dict(
        type='scatter',
        x=[fitted[relation_index,0]],
        y=[fitted[relation_index,1]],
        mode='markers',
        name=relation,
        marker=dict(
            size=12,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5),
            opacity=0.8)
    )
    data.append(trace)

layout = dict(
        xaxis=dict(title='PC1', showline=False),
        yaxis=dict(title='PC2', showline=False)
)
fig = dict(data=data, layout=layout)
iplot(fig)